## RAGASによるテスト
---

#### 1. スクレイピングによりサイト情報をテキスト（ソース元）とする
---

In [1]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://mangapedia.com/%E3%81%BC%E3%81%A3%E3%81%A1%E3%83%BB%E3%81%96%E3%83%BB%E3%82%8D%E3%81%A3%E3%81%8F%EF%BC%81-3iy4p4ji1#:~:text=%E5%8F%8B%E9%81%94%E3%81%AE%E3%81%84%E3%81%AA%E3%81%84%E5%BC%95%E3%81%A3%E8%BE%BC%E3%81%BF%E6%80%9D%E6%A1%88,%E3%82%B3%E3%83%9F%E3%82%AB%E3%83%AB%E3%81%AB%E6%8F%8F%E3%81%84%E3%81%A6%E3%81%84%E3%82%8B%E3%80%82"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
text = soup.get_text()

text = text.replace(' ', '')
text = text.replace('\n', '')

with open("bocchi.txt", 'w', encoding='utf-8') as file:
    file.write(text)

len(text)

11925

#### 2. とりあえずVectorStoreを作る
---

In [2]:
%%capture
!pip install llama-cpp-python
!pip install -qU langchain-community faiss-gpu
!pip install -qU sentence-transformers
!pip install ragas trange

In [3]:
import logging
import sys

from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain.text_splitter import CharacterTextSplitter

import tqdm, trange

In [4]:
text_splitter = CharacterTextSplitter(
    separator = "。",  # セパレータ
    chunk_size = 140,  # チャンクの文字数
    chunk_overlap = 0,  # チャンクオーバーラップの文字数
)

with open("bocchi.txt") as f:
    test_all = f.read()

texts = text_splitter.split_text(test_all)

for i in range(len(texts)):
  texts[i] = Document(page_content=texts[i], metadata={"sentence": i})


vectorstore = FAISS.from_documents(texts, HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large"))
vectorstore.save_local("vectorstore")

<ipython-input-4-a1318d04429b>:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = FAISS.from_documents(texts, HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large"))
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging 

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

#### 3.ragas
---

In [5]:
%%capture
!pip install ragas

In [6]:
!pip show ragas

Name: ragas
Version: 0.2.1
Summary: 
Home-page: 
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: appdirs, datasets, langchain, langchain-community, langchain-core, langchain-openai, nest-asyncio, numpy, openai, pydantic, pysbd, tiktoken
Required-by: 


In [7]:
from pydantic import BaseModel
from ragas.testset.graph import Node

docs = vectorstore.docstore._dict.values()
docs = list(docs)

nodes = []
for doc in docs:
  node = Node(properties={"page_content" : dict(doc).get('page_content')})
  nodes.append(node)

nodes

/usr/local/lib/python3.10/dist-packages/ragas/prompt/base.py:9: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.llms.prompt import PromptValue


[Node(id: 582b34, type: , properties: ['page_content']),
 Node(id: 38497a, type: , properties: ['page_content']),
 Node(id: e3dd46, type: , properties: ['page_content']),
 Node(id: 0b8132, type: , properties: ['page_content']),
 Node(id: 2a2c34, type: , properties: ['page_content']),
 Node(id: c56fde, type: , properties: ['page_content']),
 Node(id: 52f7de, type: , properties: ['page_content']),
 Node(id: d309ae, type: , properties: ['page_content']),
 Node(id: 547c76, type: , properties: ['page_content']),
 Node(id: e4eec8, type: , properties: ['page_content']),
 Node(id: 0a0aed, type: , properties: ['page_content']),
 Node(id: abb156, type: , properties: ['page_content']),
 Node(id: bc7527, type: , properties: ['page_content']),
 Node(id: 2ba6d2, type: , properties: ['page_content']),
 Node(id: 4538bd, type: , properties: ['page_content']),
 Node(id: 84f200, type: , properties: ['page_content']),
 Node(id: 3ef90a, type: , properties: ['page_content']),
 Node(id: 83cfef, type: , prope

In [8]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [9]:
from ragas.testset.transforms.extractors import NERExtractor

extractor = NERExtractor()
output = [await extractor.extract(node) for node in nodes]
output[0]

('entities',
 {'ORG': ['マンガペディア'], 'LOC': [], 'PER': ['まじあき'], 'MISC': ['ぼっち・ざ・ろっく!']})

In [10]:
_ = [node.properties.update({key:val}) for (key,val), node in zip(output, nodes)]
nodes[0].properties

{'page_content': 'ぼっち・ざ・ろっく!（漫画）-マンガペディアログイン誤字脱字の指摘事実相違の指摘確認画面もどるご連絡有難うございます。送信頂きました内容はすみやかに精査しコンテンツに反映致します。引き続きマンガペディアをよろしくお願い致します。ぼっち・ざ・ろっく!はまじあきの代表作',
 'entities': {'ORG': ['マンガペディア'],
  'LOC': [],
  'PER': ['まじあき'],
  'MISC': ['ぼっち・ざ・ろっく!']}}

In [11]:
import pickle
with open('nodes.pkl', 'wb') as f:
  pickle.dump(nodes, f)

In [12]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.transforms.relationship_builders.cosine import JaccardSimilarityBuilder

kg = KnowledgeGraph(nodes=nodes)
rel_builder = JaccardSimilarityBuilder(property_name="entities", key_name="PER", new_property_name="entity_jaccard_similarity")
relationships = await rel_builder.transform(kg)
relationships

[Relationship(Node(id: 582b34) <-> Node(id: 79a1aa), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(Node(id: 38497a) <-> Node(id: 2a2c34), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(Node(id: 38497a) <-> Node(id: abb156), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(Node(id: 38497a) <-> Node(id: 82216b), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(Node(id: 38497a) <-> Node(id: 417544), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(Node(id: 38497a) <-> Node(id: ba87e3), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(Node(id: 38497a) <-> Node(id: 01a1bd), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(Node(id: 38497a) <-> Node(id: e3d33c), type: jaccard_similarity, properties: ['entity_jaccard_similarity']),
 Relationship(No

In [13]:
from dataclasses import dataclass
from ragas.testset.synthesizers.base_query import QuerySynthesizer

@dataclass
class EntityQuerySynthesizer(QuerySynthesizer):

    async def _generate_scenarios( self, n, knowledge_graph, callbacks):
        """
        logic to query nodes with entity
        logic describing how to combine nodes,styles,length,persona to form n scenarios
        """

        return scenarios

    async def _generate_sample(
        self, scenario, callbacks
    ):

        """
        logic on how to use tranform each scenario to EvalSample (Query,Context,Reference)
        you may create singleturn or multiturn sample
        """

        return SingleTurnSample(user_input=query, reference_contexs=contexts, reference=reference)

In [14]:
import ragas.testset.transforms as transforms

In [15]:
print(type(extractor))
print(type(rel_builder))


<class 'ragas.testset.transforms.extractors.llm_based.NERExtractor'>
<class 'ragas.testset.transforms.relationship_builders.cosine.JaccardSimilarityBuilder'>


In [16]:
from ragas.testset.transforms import apply_transforms
transforms = [
    extractor,
    rel_builder
    ]

a = apply_transforms(kg,transforms)

Applying NERExtractor:   0%|          | 0/99 [00:00<?, ?it/s]

Applying JaccardSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
transforms

[NERExtractor(name='NERExtractor', filter_nodes=<function default_filter at 0x7d9b72c36cb0>, llm=LangchainLLMWrapper(run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=<class 'openai.RateLimitError'>, log_tenacity=False, seed=42), multiple_completion_supported=False), merge_if_possible=True, property_name='entities', prompt=NERPrompt(instruction=Extract named entities from the given text., examples=[(StringIO(text='Artificial intelligence\n\nArtificial intelligence is transforming various industries by automating tasks that previously required human intelligence. From healthcare to finance, AI is being used to analyze vast amounts of data quickly and accurately. This technology is also driving innovations in areas like self-driving cars and personalized recommendations.'), NEROutput(entities=NamedEntities(ORG=['Artificial intelligence'], LOC=['healthcare', 'finance'], PER=[], MISC=['self-driving cars', 'personalized recommendations'])))], lan

In [18]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper

llm = ChatOpenAI(
    model="gpt-4o-mini"
)

llm = LangchainLLMWrapper(llm)

In [19]:
from ragas.testset import TestsetGenerator

testset = TestsetGenerator(
    llm = llm,
    knowledge_graph = kg
)

In [20]:
testset_data = testset.generate_with_langchain_docs(
    documents = docs,
    testset_size = 10,
    with_debugging_logs=True,
    raise_exceptions=False
)

Applying [SummaryExtractor, HeadlinesExtractor]:   0%|          | 0/198 [00:00<?, ?it/s]

Applying EmbeddingExtractor:   0%|          | 0/99 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/99 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, KeyphrasesExtractor, TitleExtractor]:   0%|          | 0/375 [00:00<?, ?it/s]

Applying CosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryCosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating common_concepts:   0%|          | 0/1 [00:00<?, ?it/s]

Generating common themes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [28]:
import pandas as pd
pd.DataFrame(testset_data.to_pandas()['eval_sample'].to_dict()).to_csv('testset_data.csv')
pd.DataFrame(testset_data.to_pandas()['eval_sample'].to_dict())

,0,1,2,3,4,5,6,7,8,9,10
user_input,What conceptual frameworks can be derived from...,What conceptual frameworks can be derived from...,What conceptual frameworks can be derived from...,"How do the portrayals of band dynamics, indivi...",How do the various portrayals of music perform...,How do the character traits and narratives of ...,What role does ふりがな play in the context of ふりが...,佐藤愛子 フリーライター 結束バンド 名 知る 方法,What is the release date for the second part o...,What is the significanse of ぼっちざろっく in the con...,What role does ふりがな play in the context of ふりが...
retrieved_contexts,None,None,None,None,None,None,None,None,None,None,None
reference_contexts,"[, ]","[, ]","[, ]",[ひとりは「ギターヒーロー」として動画配信を行い、女子高生ギタリストとして人気を集めている。...,[ひとりは「ギターヒーロー」として動画配信を行い、女子高生ギタリストとして人気を集めている。...,[ひとりは「ギターヒーロー」として動画配信を行い、女子高生ギタリストとして人気を集めている。...,[ふりがなぼっちざろっく],[ひとりの奇行により、結束バンドの名はフリーライターの佐藤愛子の耳に入ることとなる。そして],[2024年6月7日に前編『劇場総集編ぼっち・ざ・ろっく!Re:』、8月9日に後編『劇場総集...,[ふりがなぼっちざろっく],[ふりがなぼっちざろっく]
response,None,None,None,None,None,None,None,None,None,None,None
multi_responses,None,None,None,None,None,None,None,None,None,None,None
reference,The absence of a defined theme or context can ...,The absence of a defined theme or context can ...,The absence of a defined theme or context can ...,"The portrayals of band dynamics, individual ch...",The various portrayals of music performance an...,The protagonists in the reports exhibit divers...,"In the context of ふりがなぼっちざろっく, ふりがな serves as ...",結束バンドの名はフリーライターの佐藤愛子の耳に入ることとなる。,The release date for the second part of the fi...,The significance of ぼっちざろっく in the context of ...,"In the context of ふりがなぼっちざろっく, ふりがな serves as ..."
rubric,None,None,None,None,None,None,None,None,None,None,None


In [23]:
testset_data.to_pandas()

,eval_sample,synthesizer_name
0,{'user_input': 'What conceptual frameworks can...,AbstractQuerySynthesizer
1,{'user_input': 'What conceptual frameworks can...,AbstractQuerySynthesizer
2,{'user_input': 'What conceptual frameworks can...,AbstractQuerySynthesizer
3,{'user_input': 'How do the portrayals of band ...,ComparativeAbstractQuerySynthesizer
4,{'user_input': 'How do the various portrayals ...,ComparativeAbstractQuerySynthesizer
5,{'user_input': 'How do the character traits an...,ComparativeAbstractQuerySynthesizer
6,{'user_input': 'What role does ふりがな play in th...,SpecificQuerySynthesizer
7,"{'user_input': '佐藤愛子 フリーライター 結束バンド 名 知る 方法', '...",SpecificQuerySynthesizer
8,{'user_input': 'What is the release date for t...,SpecificQuerySynthesizer
9,{'user_input': 'What is the significanse of ぼっ...,SpecificQuerySynthesizer


In [22]:
testset_data.to_pandas().to_csv('testset_data.csv')

In [21]:
with open('testset_data.pkl', 'wb') as f:
  pickle.dump(testset_data, f)

In [ ]:
testset_data

NameError: name 'testset_data' is not defined

In [ ]:
import inspect

# 関数の引数を取得
signature = inspect.signature(testset.generate_with_langchain_docs)
params = signature.parameters

# 引数の情報を表示
for name, param in params.items():
    print(f"引数名: {name}")
    print(f"デフォルト値: {param.default}")
    print(f"引数の型: {param.annotation}")
    print("---")

引数名: documents
デフォルト値: <class 'inspect._empty'>
引数の型: t.Sequence[LCDocument]
---
引数名: testset_size
デフォルト値: <class 'inspect._empty'>
引数の型: int
---
引数名: transforms
デフォルト値: None
引数の型: t.Optional[Transforms]
---
引数名: query_distribution
デフォルト値: None
引数の型: t.Optional[QueryDistribution]
---
引数名: run_config
デフォルト値: None
引数の型: t.Optional[RunConfig]
---
引数名: callbacks
デフォルト値: None
引数の型: t.Optional[Callbacks]
---
引数名: with_debugging_logs
デフォルト値: False
引数の型: <class 'inspect._empty'>
---
引数名: raise_exceptions
デフォルト値: True
引数の型: bool
---
